In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgbm

In [2]:
test = pd.read_csv('input/test.csv', parse_dates=['ClickDate'], 
                    dtype= { 'ID' : np.int32, 
                            'Carrier' : np.float32,                             
                             #'publisherId' : np.int32,
                            'advertiserCampaignId' : np.float32,
                            'Fraud': np.float32},
                     encoding='UTF-8')  
                   

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
test_Id = test['ID'].values
test.Country.fillna('other', inplace = True)
test.TrafficType.fillna('other', inplace = True)
test.Device.fillna('other', inplace = True)
test.Browser.fillna('other', inplace = True)
test.OS.fillna('other', inplace = True)
test.RefererUrl.fillna('other', inplace = True)
test.subPublisherId.fillna('other', inplace = True)
test['publisherId'] = pd.to_numeric(test['publisherId'], errors='coerce')
test['subPublisherId'] = pd.to_numeric(test['subPublisherId'], errors='coerce')

cols_to_encode = ['Country',  'TrafficType', 'Device','Browser', 'OS', 'RefererUrl', 'UserIp', 
                 'subPublisherId','publisherId']
le = LabelEncoder()
for col in cols_to_encode:
    test[col] = le.fit_transform(test[col])
    
test['tweekday'] = test['ClickDate'].dt.weekday
test['thour'] = test['ClickDate'].dt.hour
test['tminute'] = test['ClickDate'].dt.minute    
    
ref_url_cnt = test.groupby(['UserIp'])[ 'RefererUrl'].nunique().reset_index().\
                            rename(columns = {'RefererUrl': 'ref_url_cnt'})
pubId_cnt = test.groupby(['UserIp'])[ 'publisherId'].nunique().reset_index().\
                        rename(columns = {'publisherId': 'pubId_cnt'})
adv_comp_cnt = test.groupby(['UserIp'])[ 'advertiserCampaignId'].nunique().reset_index().\
                             rename(columns = {'advertiserCampaignId': 'compId_cnt'})
    
test = pd.merge(test, ref_url_cnt, how='left', on=['UserIp'])
test = pd.merge(test, pubId_cnt, how='left', on=['UserIp'])
test = pd.merge(test, adv_comp_cnt, how='left', on=['UserIp'])
del ref_url_cnt, pubId_cnt, adv_comp_cnt

test['refUrl_user_cnt'] = test.groupby(['RefererUrl'])['ID'].transform('count')
test['user_cnt'] = test.groupby(['UserIp'])['ID'].transform('count')
test['user_week_cnt'] = test.groupby(['UserIp', 'tweekday'])['ID'].transform('count')
test['user_hr_cnt'] = test.groupby(['UserIp', 'tweekday', 'thour'])['ID'].transform('count')

test['user_refurl_share'] = test['user_cnt']/ test['ref_url_cnt']
test['user_pubid_share'] = test['user_cnt']/ test['pubId_cnt']
test['user_compid_share'] = test['user_cnt']/test['compId_cnt']

test.drop(['ID','ClickDate'], axis=1, inplace=True)

In [4]:
test.shape

(25548873, 25)

chunksize = 10 ** 7
#chunks = pd.read_csv('input/train.csv', chunksize=chunksize)
chunks = pd.read_csv('input/train.csv', parse_dates=['ClickDate'], 
                    dtype= { 'ID' : np.int32, 
                            'Carrier' : np.float32, 
                            'ConversionPayOut' : np.float32,
                             #'publisherId' : np.int32,
                            'advertiserCampaignId' : np.float32,
                            'Fraud': np.float32},
                     encoding='UTF-8',  
                     chunksize=chunksize)

In [5]:
i = 1
y_preds = pd.DataFrame()
#chunksize = 10 ** 7
chunksize = 21122405
for train in  pd.read_csv('input/train.csv', iterator=True, chunksize=chunksize):
    #len_train = train.shape[0]
    y_train = train.ConversionPayOut.values
    train.drop(['ConversionStatus','ConversionDate', 'ConversionPayOut'], inplace = True, axis=1)
    
    train.Country.fillna('other', inplace = True)
    train.TrafficType.fillna('other', inplace = True)
    train.Device.fillna('other', inplace = True)
    train.Browser.fillna('other', inplace = True)
    train.OS.fillna('other', inplace = True)
    train.RefererUrl.fillna('other', inplace = True)
    train.subPublisherId.fillna('other', inplace = True)
    train['publisherId'] = pd.to_numeric(train['publisherId'], errors='coerce')
    train['subPublisherId'] = pd.to_numeric(train['subPublisherId'], errors='coerce')
    
    cols_to_encode = ['Country',  'TrafficType', 'Device','Browser', 'OS', 'RefererUrl', 'UserIp', 
                 'subPublisherId','publisherId']
    le = LabelEncoder()
    for col in cols_to_encode:
        train[col] = le.fit_transform(train[col])
        
    train['ClickDate'] = pd.to_datetime(train['ClickDate'])
    train['tweekday'] = train['ClickDate'].dt.weekday
    train['thour'] = train['ClickDate'].dt.hour
    train['tminute'] = train['ClickDate'].dt.minute
    
    ref_url_cnt = train.groupby(['UserIp'])[ 'RefererUrl'].nunique().reset_index().\
                            rename(columns = {'RefererUrl': 'ref_url_cnt'})
    pubId_cnt = train.groupby(['UserIp'])[ 'publisherId'].nunique().reset_index().\
                        rename(columns = {'publisherId': 'pubId_cnt'})
    adv_comp_cnt = train.groupby(['UserIp'])[ 'advertiserCampaignId'].nunique().reset_index().\
                             rename(columns = {'advertiserCampaignId': 'compId_cnt'})
        
    train = pd.merge(train, ref_url_cnt, how='left', on=['UserIp'])
    train = pd.merge(train, pubId_cnt, how='left', on=['UserIp'])
    train = pd.merge(train, adv_comp_cnt, how='left', on=['UserIp'])    
    del ref_url_cnt, pubId_cnt, adv_comp_cnt     
    
    train['refUrl_user_cnt'] = train.groupby(['RefererUrl'])['ID'].transform('count')
    train['user_cnt'] = train.groupby(['UserIp'])['ID'].transform('count')
    train['user_week_cnt'] = train.groupby(['UserIp', 'tweekday'])['ID'].transform('count')
    train['user_hr_cnt'] = train.groupby(['UserIp', 'tweekday', 'thour'])['ID'].transform('count')
    
    train['user_refurl_share'] = train['user_cnt']/ train['ref_url_cnt']
    train['user_pubid_share'] = train['user_cnt']/ train['pubId_cnt']
    train['user_compid_share'] = train['user_cnt']/train['compId_cnt']
    
    train.drop(['ID','ClickDate'], axis=1, inplace=True)
    
    train_X, valid_X, train_y, valid_y = train_test_split(train, y_train, test_size=0.3, random_state=32)
    del train, y_train
    
    lgb_train = lgbm.Dataset(train_X, train_y)
    lgb_eval = lgbm.Dataset(valid_X, valid_y, reference=lgb_train)    
    del train_X, valid_X, train_y, valid_y
    
    params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.02,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    #'device':'gpu',
    'verbose': 10
    }
    
    
    print('training model: {}'.format(i))
    gbm = lgbm.train(params,
                lgb_train,                
                num_boost_round=1000,
                valid_sets=lgb_eval,
                early_stopping_rounds=50)
    
    y_preds['gbm'+ str(i)] = gbm.predict(test, num_iteration=gbm.best_iteration)
    del lgb_train,lgb_eval    
    i += 1

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


training model: 1
[1]	valid_0's rmse: 6.23309
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's rmse: 6.2331
[3]	valid_0's rmse: 6.2331
[4]	valid_0's rmse: 6.23311
[5]	valid_0's rmse: 6.23308
[6]	valid_0's rmse: 6.23309
[7]	valid_0's rmse: 6.23311
[8]	valid_0's rmse: 6.23312
[9]	valid_0's rmse: 6.23315
[10]	valid_0's rmse: 6.23317
[11]	valid_0's rmse: 6.23312
[12]	valid_0's rmse: 6.23314
[13]	valid_0's rmse: 6.23318
[14]	valid_0's rmse: 6.23323
[15]	valid_0's rmse: 6.23326
[16]	valid_0's rmse: 6.2333
[17]	valid_0's rmse: 6.23334
[18]	valid_0's rmse: 6.23336
[19]	valid_0's rmse: 6.23332
[20]	valid_0's rmse: 6.23336
[21]	valid_0's rmse: 6.23338
[22]	valid_0's rmse: 6.23341
[23]	valid_0's rmse: 6.23345
[24]	valid_0's rmse: 6.23347
[25]	valid_0's rmse: 6.23352
[26]	valid_0's rmse: 6.23354
[27]	valid_0's rmse: 6.23356
[28]	valid_0's rmse: 6.23359
[29]	valid_0's rmse: 6.23361
[30]	valid_0's rmse: 6.23364
[31]	valid_0's rmse: 6.23365
[32]	valid_0's rmse: 6.23369
[33]

TypeError: 'NoneType' object is not iterable

In [6]:
y_preds.head(5)

,gbm1
0,0.024941
1,0.024052
2,0.025618
3,0.023564
4,0.023746


In [8]:
sub = pd.DataFrame({'ID': test_Id, 'ConversionPayOut' : y_preds.gbm1.values})
sub = sub[['ID','ConversionPayOut'] ]

In [10]:
sub.head(5)

,ID,ConversionPayOut
0,63367289,0.024941
1,63367290,0.024052
2,63367291,0.025618
3,63367292,0.023564
4,63367293,0.023746


In [11]:
sub.to_csv('lgbm_v4.csv', index=False)